# Dataanalyse
Denne notebooken inneholder analyse av det sammenslåtte datasettet med miljødata fra NASA og Frost. Vi starter med å laste inn data, gjøre statistiske analyser, identifisere mulige sammenhenger i datasettet og vise de frem visuelt med grafer og scattershot.

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose


# Les direkte fra sti
df = pd.read_csv("../data/clean/merged_data.csv")
# Konverter 'Date'-kolonnen til datetime
df['Date'] = pd.to_datetime(df['Date'])
# Sorter og reset index
df = df.sort_values('Date').reset_index(drop=True)

# Sjekk format og første rader
df.info()
df.head()



In [ ]:
# Fjern irrelevante kolonner
exclude = ['Unnamed:_0', 'Wind_Direction']
data_cols = [col for col in df.columns if col not in exclude + ['Date']]

# Statistisk oppsummering
print("### Statistisk oppsummering per variabel ###\n")
for col in data_cols:
    print(f"--- {col} ---")
    print(f"  Gjennomsnitt:     {df[col].mean():.2f}")
    print(f"  Median:           {df[col].median():.2f}")
    print(f"  Standardavvik:    {df[col].std():.2f}")
    print(f"  Varians:          {df[col].var():.2f}")
    print(f"  Min:              {df[col].min():.2f}")
    print(f"  Maks:             {df[col].max():.2f}\n")

# Ekstra analyse: Årlige og månedlige gjennomsnitt for temperatur
if 'Temperature' in df.columns:
    print("### Årlige gjennomsnittstemperaturer ###")
    df['Year'] = df['Date'].dt.year
    yearly_temp = df.groupby('Year')['Temperature'].mean()
    print(yearly_temp.round(2).to_string())
    
    print("\n### Månedlige gjennomsnittstemperaturer ###")
    monthly_avg_temp = df.groupby(df['Date'].dt.to_period('M'))['Temperature'].mean()
    print(monthly_avg_temp.round(2).to_string())


In [ ]:
# enkel korrelasjonsanalyse

def correlation_analysis(col1, col2):
    r, p = stats.pearsonr(df[col1].dropna(), df[col2].dropna())
    print(f"Pearson korrelasjon ({col1} vs {col2}): r = {r:.3f}, p = {p:.3e}")
    return r, p

# Eksempel: Temperatur vs TS
ec1, p1 = correlation_analysis('Temperature', 'TS') if 'TS' in df.columns else (None, None)


In [ ]:
# Sørg for at 'Date' er i datetime-format
df['Date'] = pd.to_datetime(df['Date'])

# Miljøvariabler for analyse
miljøkolonner = ['Temperature', 'Humidity', 'Pressure', 'Precipitation', 'Wind_Speed',
                 'ALLSKY_SFC_SW_DWN', 'TS', 'QV2M', 'PS']

# Histogrammer for tre utvalgte variabler
fig, axs = plt.subplots(1, 3, figsize=(18, 5))

df['Precipitation'].hist(ax=axs[0], bins=40, edgecolor='black')
axs[0].set_title("Fordeling av nedbør (mm)")
axs[0].set_xlabel("Nedbør")
axs[0].set_ylabel("Antall dager")

df['Wind_Speed'].hist(ax=axs[1], bins=40, edgecolor='black')
axs[1].set_title("Fordeling av vindstyrke (m/s)")
axs[1].set_xlabel("Vindstyrke")

df['Pressure'].hist(ax=axs[2], bins=40, edgecolor='black')
axs[2].set_title("Fordeling av lufttrykk (hPa)")
axs[2].set_xlabel("Trykk")

plt.tight_layout()
plt.show()

# Legg til månedskolonne
df['Month'] = df['Date'].dt.month

# Månedlig gjennomsnitt
monthly_avg = df.groupby('Month')[['Temperature', 'Humidity', 'Precipitation', 'Wind_Speed', 'ALLSKY_SFC_SW_DWN']].mean()

# Månedlig standardavvik (for å se variasjon)
monthly_std = df.groupby('Month')[['Temperature', 'Humidity', 'Precipitation', 'Wind_Speed', 'ALLSKY_SFC_SW_DWN']].std()

# Plot månedlig gjennomsnitt
plt.figure(figsize=(12, 6))
monthly_avg.plot(marker='o', title="Månedlig gjennomsnitt av miljøvariabler", ax=plt.gca())
plt.xlabel("Måned")
plt.ylabel("Gjennomsnittsverdi")
plt.grid(True)
plt.tight_layout()
plt.show()

# Plot månedlig standardavvik
plt.figure(figsize=(12, 6))
monthly_std.plot(marker='x', linestyle='--', title="Månedlig variasjon (std) av miljøvariabler", ax=plt.gca())
plt.xlabel("Måned")
plt.ylabel("Standardavvik")
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
# Sørg for korrekt formatering
df = pd.read_csv("../data/clean/merged_data.csv")
df['Date'] = pd.to_datetime(df['Date'])
df = df.set_index('Date').sort_index()

# Kolonner vi vil analysere
vars_available = [col for col in df.columns if col != 'Unnamed:_0']

# Decompose hver variabel
for col in vars_available:
    series = df[col].copy()

    # Hopp over hvis for få datapunkter
    if series.dropna().shape[0] < 730:  # 2 år som minimum for decomposing
        print(f"Skipper {col} – for lite data.")
        continue

    # Fjern outliers for ALLSKY_SFC_SW_DWN
    if col == 'ALLSKY_SFC_SW_DWN':
        q1 = series.quantile(0.25)
        q3 = series.quantile(0.75)
        iqr = q3 - q1
        upper = q3 + 1.5 * iqr
        lower = q1 - 1.5 * iqr
        series = series[(series >= lower) & (series <= upper)]

    # Interpoler og sett frekvens
    daily = series.asfreq('D').interpolate()

    # Kjør dekomponering
    try:
        decomp = seasonal_decompose(daily, model='additive', period=365)
        fig = decomp.plot()
        fig.set_size_inches(12, 8)
        plt.suptitle(f"Sesongdekomponering – {col}", fontsize=16, y=1.02)
        plt.tight_layout()
        plt.show()
    except ValueError as e:
        print(f"Kunne ikke dekomponere {col}: {e}")


In [ ]:
corr = df[vars_available].corr()
plt.figure(figsize=(8,6))
sns.heatmap(corr, annot=True, fmt='.2f', cmap='vlag', center=0)
plt.title('Korrelasjonsmatrise')
plt.show()